In [1]:
import tensorflow as tf

D:\anaconda3\julianxu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
csv = [
  "1,harden|james|curry",
  "2,wrestbrook|harden|durant",
  "3,|paul|towns",
]

In [3]:
TAG_SET = ["harden", "james", "curry", "durant", "paul","towns","wrestbrook"]

In [4]:
# 处理得到SpareTensor
ids,post_tags_str = tf.decode_csv(csv,[[-1],[""]])

In [7]:
table = tf.contrib.lookup.index_table_from_tensor(
mapping=TAG_SET,default_value=-1) # 构造一个查找表

In [9]:
split_tags = tf.string_split(post_tags_str,"|")

In [12]:
split_tags.indices

<tf.Tensor 'StringSplit:0' shape=(?, 2) dtype=int64>

In [14]:
tags = tf.SparseTensor(
indices = split_tags.indices,
values = table.lookup(split_tags.values),
    dense_shape=split_tags.dense_shape)

In [15]:
# 定义embedding变量
# 大小为3 因为 只有7个类型
TAG_EMBEDDING_DIM = 3
embedding_params = tf.Variable(tf.truncated_normal([len(TAG_SET),TAG_EMBEDDING_DIM]))

In [16]:
embedding_tags = tf.nn.embedding_lookup_sparse(embedding_params,sp_ids=tags,sp_weights=None)
# sp_ids就是我们刚刚得到的SparseTensor，而sp_weights=None代表的每一个取值的权重，如果是None的话，所有权重都是1，也就是相当于取了平均
# 如果不是None的话，我们需要同样传入一个SparseTensor，代表不同球员的喜欢权重。大家感兴趣可以自己去尝试

In [18]:
with tf.Session() as s:
    s.run([tf.global_variables_initializer(),tf.tables_initializer()])
    print(s.run([tags,embedding_tags,post_tags_str]))

[SparseTensorValue(indices=array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2],
       [2, 0],
       [2, 1]], dtype=int64), values=array([0, 1, 2, 6, 0, 3, 4, 5], dtype=int64), dense_shape=array([3, 3], dtype=int64)), array([[ 0.06023904,  1.0575624 , -0.9093878 ],
       [-0.42566654,  0.26845995, -0.6602178 ],
       [-0.6277443 ,  0.28916246, -0.15512544]], dtype=float32), array([b'harden|james|curry', b'wrestbrook|harden|durant', b'|paul|towns'],
      dtype=object)]
